In [2]:
!pip install nolds
!pip install hmmlearn
!pip install yfinance
!pip install statsmodels

In [3]:
import yfinance as yf
import datetime
from numpy import *
from pylab import plot, show

import pandas as pd
import numpy as np

import numpy as np
from pathlib import Path
from hmmlearn.hmm import GaussianHMM
from sklearn.preprocessing import StandardScaler
#from google.colab import drive

In [17]:
#Umcomment this method when running from google colab
#def load_data_from_drive(start_date, end_date):
  #drive.mount('/content/drive', force_remount=True)
  #base_path = Path("/content/drive/My Drive/WQU/Capstone/data/parquet_out")
  #dfs = []
  #for d in sorted(base_path.iterdir()):
      #if d.is_dir() and start_date <= d.name <= end_date:
          #dfs.append(pd.read_parquet(d))

  #return pd.concat(dfs, ignore_index=True)

def load_data(start_date, end_date):
  base_path = Path("./data/parquet_out")
  dfs = []
  for d in sorted(base_path.iterdir()):
      if d.is_dir() and start_date <= d.name <= end_date:
          dfs.append(pd.read_parquet(d))

  return pd.concat(dfs, ignore_index=True)




In [5]:
df = load_data("2026-01-01", "2026-01-12")
print(df.head())
print(df.tail())

                        time    symbol bidqty askqty       ltp   atp   ask  \
0  2026-01-01T09:15:00+05:30  NIFTY 50      0      0  26183.50  0.00  0.00   
1  2026-01-01T09:16:00+05:30  NIFTY 50      0      0  26194.30  0.00  0.00   
2  2026-01-01T09:17:00+05:30  NIFTY 50      0      0  26189.55  0.00  0.00   
3  2026-01-01T09:18:00+05:30  NIFTY 50      0      0  26179.50  0.00  0.00   
4  2026-01-01T09:19:00+05:30  NIFTY 50      0      0  26167.80  0.00  0.00   

    bid oi tot_vol tot_buyqty tot_sellqty ltq                  ltt level2  \
0  0.00  0       0          0           0   0  2026-01-01 09:15:00   None   
1  0.00  0       0          0           0   0  2026-01-01 09:16:00   None   
2  0.00  0       0          0           0   0  2026-01-01 09:17:00   None   
3  0.00  0       0          0           0   0  2026-01-01 09:18:00   None   
4  0.00  0       0          0           0   0  2026-01-01 09:19:00   None   

  recv_time bcast_time security_id  
0         0       None      256

**Function to get 1 min data**

In [6]:
def fill_intraday_minutes(day_df):
    day = day_df.index[0].date()
    full_index = pd.date_range(
        start=pd.Timestamp(f"{day} 09:15"),
        end=pd.Timestamp(f"{day} 15:30"),
        freq="1min"
    )
    return day_df.reindex(full_index).ffill()

def get_data_one_min_freq(start_date, end_date):
  df = load_data(start_date=start_date, end_date=end_date)
  df.drop(columns=['time', 'symbol', 'security_id', 'level2', 'bcast_time'], inplace=True)
  df['ltt'] = pd.to_datetime(df['ltt'])
  df = df.sort_values('ltt').set_index('ltt')
  df_1min = (
    df
    .groupby(df.index.date, group_keys=False)
    .apply(fill_intraday_minutes)
  )
  return df_1min



In [7]:
df_1min = get_data_one_min_freq("2026-01-01", "2026-01-12")
print(df_1min.index.to_series().diff().value_counts())
print(df_1min.groupby(df_1min.index.date).size().value_counts())

0 days 00:01:00    3000
0 days 17:45:00       5
2 days 17:45:00       2
Name: count, dtype: int64
376    8
Name: count, dtype: int64


**Function to get 5 mins data**

In [8]:
def fill_intraday_5min(day_df):
    day = day_df.index[0].date()

    # Create fixed 5-min intraday grid
    full_index = pd.date_range(
        start=pd.Timestamp(f"{day} 09:15"),
        end=pd.Timestamp(f"{day} 15:30"),
        freq="5min"
    )

    # Resample to 5-min bars (last price per bar)
    day_5min = (
        day_df
        .resample("5min")
        .last()
    )

    # Reindex to full grid and forward-fill within day
    return day_5min.reindex(full_index).ffill()


def get_data_5min_freq(start_date, end_date):
    df = load_data(start_date=start_date, end_date=end_date)

    df.drop(
        columns=['time', 'symbol', 'security_id', 'level2', 'bcast_time'],
        inplace=True
    )

    df['ltt'] = pd.to_datetime(df['ltt'])
    df = df.sort_values('ltt').set_index('ltt')

    df = df[~df.index.isna()]

    df_5min = (
        df
        .groupby(df.index.date, group_keys=False)
        .apply(fill_intraday_5min)
    )

    return df_5min


In [9]:
df_5min = get_data_5min_freq("2026-01-01", "2026-01-12")
print(df_5min.index.to_series().diff().value_counts())
print(df_5min.groupby(df_5min.index.date).size().value_counts())

0 days 00:05:00    600
0 days 17:45:00      5
2 days 17:45:00      2
Name: count, dtype: int64
76    8
Name: count, dtype: int64


In [10]:
df_5min.groupby(df_5min.index.date).size().head()

2026-01-01    76
2026-01-02    76
2026-01-05    76
2026-01-06    76
2026-01-07    76
dtype: int64

**Function to get 15 mins data**

In [11]:
def fill_intraday_15min(day_df):
    day = day_df.index[0].date()

    # Create fixed 15-min intraday grid
    full_index = pd.date_range(
        start=pd.Timestamp(f"{day} 09:15"),
        end=pd.Timestamp(f"{day} 15:30"),
        freq="15min"
    )

    # Resample to 5-min bars (last price per bar)
    day_15min = (
        day_df
        .resample("15min")
        .last()
    )

    # Reindex to full grid and forward-fill within day
    return day_15min.reindex(full_index).ffill()


def get_data_15min_freq(start_date, end_date):
    df = load_data(start_date=start_date, end_date=end_date)

    df.drop(
        columns=['time', 'symbol', 'security_id', 'level2', 'bcast_time'],
        inplace=True
    )

    df['ltt'] = pd.to_datetime(df['ltt'])
    df = df.sort_values('ltt').set_index('ltt')

    df = df[~df.index.isna()]

    df_15min = (
        df
        .groupby(df.index.date, group_keys=False)
        .apply(fill_intraday_15min)
    )

    return df_15min


In [12]:
df_15min = get_data_15min_freq("2026-01-01", "2026-01-12")
print(df_15min.index.to_series().diff().value_counts())
print(df_15min.groupby(df_15min.index.date).size().value_counts())

0 days 00:15:00    200
0 days 17:45:00      5
2 days 17:45:00      2
Name: count, dtype: int64
26    8
Name: count, dtype: int64


In [13]:
df_15min.tail()

,bidqty,askqty,ltp,atp,ask,bid,oi,tot_vol,tot_buyqty,tot_sellqty,ltq,recv_time
2026-01-12 14:30:00,0,0,25760.80,0.00,0.00,0.00,0,0,0,0,0,0
2026-01-12 14:45:00,0,0,25810.45,0.00,0.00,0.00,0,0,0,0,0,0
2026-01-12 15:00:00,0,0,25778.50,0.00,0.00,0.00,0,0,0,0,0,0
2026-01-12 15:15:00,0,0,25806.10,0.00,0.00,0.00,0,0,0,0,0,0
2026-01-12 15:30:00,0,0,25806.10,0.00,0.00,0.00,0,0,0,0,0,0


In [ ]:
df_15min.groupby(df_15min.index.date).size().head()

,0
2026-01-01,26
2026-01-02,26
2026-01-05,26
2026-01-06,26
2026-01-07,26


Function to get 60 mins data

In [14]:
NUMERIC_COLS = [
    "tot_vol", "ask", "bid", "oi", "askqty", "bidqty",
    "tot_buyqty", "tot_sellqty", "ltq", "atp"
]

for col in NUMERIC_COLS:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")



PRICE_COL = "ltp"
VOLUME_COL = "tot_vol"

LAST_COLS = [
    "ask", "bid", "oi", "askqty", "bidqty",
    "tot_buyqty", "tot_sellqty", "ltq", "atp", "recv_time"
]



def build_agg_dict(df):
  agg = {
      PRICE_COL: ["first", "max", "min", "last"],
      VOLUME_COL: "sum"
  }

  for col in LAST_COLS:
    if col in df.columns:
      agg[col] = "last"
  return agg


def get_data_60min_freq(start_date, end_date):
    df = load_data(start_date=start_date, end_date=end_date)

    df.drop(
        columns=['time', 'symbol', 'security_id', 'level2', 'bcast_time'],
        inplace=True
    )


    NUMERIC_COLS = [
        "tot_vol", "ask", "bid", "oi",
        "askqty", "bidqty", "tot_buyqty",
        "tot_sellqty", "ltq", "atp"
    ]

    for col in NUMERIC_COLS:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    df['ltt'] = pd.to_datetime(df['ltt'], errors="coerce")
    df = df.sort_values('ltt').set_index('ltt')
    df = df[~df.index.isna()]

    df_60min = (
        df
        .groupby(df.index.date, group_keys=False)
        .apply(fill_intraday_60min)
    )

    return df_60min


def fill_intraday_60min(day_df):
    day = day_df.index[0].date()

    full_index = pd.date_range(
        start=pd.Timestamp(f"{day} 09:15"),
        end=pd.Timestamp(f"{day} 15:15"),
        freq="60min"
    )

    agg_dict = build_agg_dict(day_df)

    day_60min = (
    day_df
    .resample(
        "60min",
        origin=pd.Timestamp(f"{day} 09:15"),
        label="right",
        closed="right"
    )
    .agg(agg_dict)
    )

    day_60min.columns = [
        f"{c[0]}_{c[1]}" if isinstance(c, tuple) else c
        for c in day_60min.columns
    ]

    return day_60min.reindex(full_index).ffill()



In [15]:
df_60min = get_data_60min_freq("2026-01-01", "2026-01-12")
print(df_60min.index.to_series().diff().value_counts())
print(df_60min.groupby(df_60min.index.date).size().value_counts())

0 days 01:00:00    48
0 days 18:00:00     5
2 days 18:00:00     2
Name: count, dtype: int64
7    8
Name: count, dtype: int64


In [16]:
df_60min.index.name = 'time'
df_60min.tail()

,ltp_first,ltp_max,ltp_min,ltp_last,tot_vol_sum,ask_last,bid_last,oi_last,askqty_last,bidqty_last,tot_buyqty_last,tot_sellqty_last,ltq_last,atp_last,recv_time_last
time,,,,,,,,,,,,,,,
2026-01-12 11:15:00,25550.40,25595.30,25528.80,25528.80,0,0.0,0.0,0,0,0,0,0,0,0.0,0
2026-01-12 12:15:00,25526.15,25578.30,25475.00,25577.45,0,0.0,0.0,0,0,0,0,0,0,0.0,0
2026-01-12 13:15:00,25583.40,25732.20,25583.40,25725.15,0,0.0,0.0,0,0,0,0,0,0,0.0,0
2026-01-12 14:15:00,25726.60,25751.55,25689.05,25729.15,0,0.0,0.0,0,0,0,0,0,0,0.0,0
2026-01-12 15:15:00,25734.05,25812.45,25705.70,25784.05,0,0.0,0.0,0,0,0,0,0,0,0.0,0
